In [ ]:
from IPython.display import Image
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns; sns.set()
import sklearn
import pandas as pd

#references https://jakevdp.github.io/PythonDataScienceHandbook/05.07-support-vector-machines.html
#https://arxiv.org/pdf/1710.05398

### Super-Earths v.s Mini-Neptunes
A surprising recent result in the exoplanet field is that the radii of small planets have a bimodal distribution. There is a lower peak at 1.5 R_earth (Super-Earths) and a higher peak at about 2.5 R_earth (Mini-Neptunes).

There is a valley (a deficit) of planets between Super-Earths and Mini-Neptunes at about 1.8 R_earth.

Further observations reveal that Super-Earths are mostly bare rocks whereas Mini-Neptunes have substantial H/He envelopes

In [ ]:
Image(url= "data/fulton_gap.png")
#http://ui.adsabs.harvard.edu/abs/2017AJ....154..109F/abstract

In [ ]:
Image(url= "data/se-mn.jpg")

### Radius Valley vs Orbital Period
It was pointed that the radius valley separating the Super-Earths and Mini-Neptunes may have a dependence on orbital periods: atmospheric erosion of H/He envelope likely depends on orbital period.

Some suggest that it could be a power law dependence: $R_{\rm p, valley} \propto P^\alpha$

In [ ]:
newdata = pd.read_csv('data/se-mn.csv',header=0)
print(newdata)
rp=np.array(newdata['Rp'],'d')
rp_unc=np.array(newdata['Rp_unc'],'d')
per=np.array(newdata['Period'],'d')
se_mn=np.array(newdata['SE_MN'],'i')
plt.errorbar(per[se_mn==-1],rp[se_mn==-1],yerr=rp_unc[se_mn==-1], color = 'purple', fmt = 'o')
plt.errorbar(per[se_mn==1],rp[se_mn==1],yerr=rp_unc[se_mn==1], color = 'red', fmt = 'o')
plt.yscale('log')
plt.xscale('log')

plt.xlabel('Orbital Periods (day)')
plt.ylabel('Planet Radius ($R_\oplus$)')
plt.ylim(0.8,6)
plt.xlim(1,110)

### Radius Valley using SVM
Let us determine how the Radius Valley depends on the orbital distance using Support Vector Machine

In [ ]:
from sklearn.svm import SVC # "Support vector classifier"
model = SVC(kernel='linear', C=1E10) #let's use a simple linear boundary


X=np.vstack((np.log10(per), np.log10(rp))).T
y = se_mn
model.fit(X, y)

In [ ]:
def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model.decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    
    # plot support vectors
    if plot_support:
        ax.scatter(model.support_vectors_[:, 0],
                   model.support_vectors_[:, 1],
                   s=300, linewidth=1, facecolors='none');
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='rainbow')
plot_svc_decision_function(model)
plt.ylim(np.log10(0.8),np.log10(6))
plt.xlim(np.log10(1),np.log10(110))

plt.xlabel('log (P/day)')
plt.ylabel('log ($R_p/R_\oplus$)')

# Exercise 1:
### Given that the weight vector w is of the SVM is:

print(model.coef_)

### how does the radius valley depend on the orbital period i.e. find alpha $R_{\rm p, valley} \propto P^\alpha$? (hint: the weight vector is the normal of the hyperplane separating the categories)

# Exercise 2:
### Using the bootstrap method and the reported uncertainty on radii of the planets, find the confidence interval of $\alpha$. 

### Can you rule out $\alpha=0$, i.e. no period dependence?

